In [10]:
"""
Created on Thu Mar 21 18:31:13 2024

@author: jordanrattle
"""
import geopandas as gpd
import folium

# URL of the GeoJson file containing the Swedish sunlight hours data (Tabulated in Google Sheets & converted into GeoJson in QGIS)
sverige_soltid = 'https://raw.githubusercontent.com/jordanrattle/Swedish-Sunlight-Hours-Map/main/Sverige_Soltid.geojson'

# List of colours for creating a colour ramp. I created this colour ramp @ https://www.rampgenerator.com/
colours = ['#FFFFE6','#FFFFE6','#FFFFE6','#FDFEBF','#FDFEBF','#FDFEBF','#FCFD99'
          ,'#FCFD99','#FCFD99','#FBFD73','#FBFD73','#FBFD73','#F9FC4C','#F9FC4C'
          ,'#F9FC4C','#F8FB26','#F8FB26','#F8FB26','#F7FB00','#F7FB00','#F7FB00']

# Read the GeoJson file in as a GeoDataFrame
soltid_gdf = gpd.read_file(sverige_soltid)

# Compute the range between the maximum and minimum annual sunlight hours
soltid_gdf['Range'] = soltid_gdf['Max since start year'] - soltid_gdf['Min since start year']

# Sort the values by the average annual sunlight hours between 1991-2020
soltid_gdf = soltid_gdf.sort_values(by='Normal 1991-20')

# Create an id column for assigning the colour ramp to each observation station's value
soltid_gdf['id'] = [x for x in range(21)]

# Create a GeoJsonTooltip class that defines the parameters for a pop-up window to emerge when hovering over features in the map
tooltip = folium.GeoJsonTooltip(
    fields=['Station', 'Normal 1991-20', 'Range'],
    aliases=['Station:', 'Average 1991-2020 [hr]:', 'Range [hr]:'],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

# Create a Map class that has its centre location and zoom specified. The basemap style has also been defined as "cartodb positron"
m = folium.Map(location=(62.6, 16.35), zoom_start=4.5,tiles="cartodb positron")

# Create a GeoJson class of using the GeoDataFrame created above
folium.GeoJson(
    soltid_gdf,
    name="Soltid",
    marker=folium.Circle(radius=4, fill_color="yellow", fill_opacity=0.5, color="grey", weight=1),
    style_function=lambda x: {
        "fillColor": colours[x['properties']['id']],
        "radius": (x['properties']['Normal 1991-20'])**2/100,
    },
    highlight_function=lambda x: {"fillOpacity": 0.9},
    tooltip = tooltip,
    zoom_on_click=True,
).add_to(m)

# Save the map as HTML file, you can change this to where you would like to save it
m.save("soltid_karta.html")

# Show the map
m